In [78]:
setwd(paste0(Sys.getenv('ROOT'), '/R'))
getwd()

[1] "/mnt/hdd/develop/kaggle/house-prices/R/R"

In [79]:
library(Metrics)
source('main.R')

data <- kaggle.house$loadData()

df.combined <- kaggle.house$getCombinedDataset(data$train, data$test) %>% (kaggle.house$na$fixAll)

df.training <- 
    df.combined %>% 
    filter(dataSource == "train") %>% 
    mutate(sale_price_log = log(SalePrice)) %>%
    select(-dataSource)

In [88]:
partition <- caret::createDataPartition(y=df.training$sale_price_log, p=.5, list=F, times=1)

trainset <- df.training[partition,] %>% select(-SalePrice)

testset <- df.training[-partition,] %>% select(-sale_price_log)

setdiff(trainset %>% colnames, testset %>% colnames)
setdiff(testset %>% colnames, trainset %>% colnames)

[1] "sale_price_log"

[1] "SalePrice"

In [89]:
tran.res <- kaggle.house$trans$doItAll(trainset, testset %>% select(-SalePrice))
trainset.ready <- tran.res$df.training
testset.ready <- tran.res$df.testing

# Should be only numeric columns
stopifnot(0 == trainset.ready %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)
stopifnot(0 == testset.ready %>% purrr::map(function (col) { !is.numeric(col) }) %>% unlist %>% sum)

c(trainset.ready %>% nrow, testset.ready %>% nrow)

c('1. ', setdiff(trainset.ready %>% colnames, testset.ready %>% colnames))
c('2. ', setdiff(testset.ready %>% colnames, trainset.ready %>% colnames))

[1] 731 729

[1] "1. "            "sale_price_log"

[1] "2. "

In [90]:
# sanity check

testset %>% group_by(BldgType) %>% summarise(median(log(SalePrice)))

cbind(testset.ready %>% select(BldgType.new), testset %>% select(BldgType)) %>% 
    group_by(BldgType) %>% summarise(min(BldgType.new), max(BldgType.new))


BldgType,median(log(SalePrice))
1Fam,12.02575
2fmCon,11.71587
Duplex,11.82591
Twnhs,11.89613
TwnhsE,12.05641


BldgType,min(BldgType.new),max(BldgType.new)
1Fam,12.03913,12.03913
2fmCon,11.81226,11.81226
Duplex,11.73407,11.73407
Twnhs,11.77529,11.77529
TwnhsE,12.05597,12.05597


In [92]:
model.lm <- lm(sale_price_log ~ ., data=trainset.ready)

y_predicted <- predict(model.lm, testset.ready) %>% as.vector
y_actual <- testset %>% mutate(sale_price_log = log(SalePrice)) %>% select(sale_price_log) %>% `[[`(1)
rmse(y_predicted, y_actual)

sp_predicted <- y_predicted %>% exp
sp_actual <- testset %>% select(SalePrice) %>% `[[`(1)
mean(100 * abs(sp_actual - sp_predicted) / sp_actual)

glance(model.lm)
tidy(model.lm) %>% arrange(p.value) %>% filter(p.value < 0.1)

Warning message in predict.lm(model.lm, testset.ready):
“prediction from a rank-deficient fit may be misleading”

[1] 0.1746755

[1] 11.56607

r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
0.9370307,0.9297132,0.1058743,128.0528,0,77,644.9006,-1133.801,-775.4369,7.330931,654


term,estimate,std.error,statistic,p.value
X2ndFlrSF,2.534787e-04,2.346005e-05,10.804699,3.757998e-25
OverallQual,5.464718e-02,6.297153e-03,8.678078,3.186243e-17
X1stFlrSF,2.397066e-04,2.854763e-05,8.396724,2.846766e-16
OverallCond,4.067998e-02,5.254481e-03,7.741960,3.732257e-14
MSZoning.new,2.519823e-01,3.738822e-02,6.739616,3.489388e-11
BsmtFinSF1,1.664376e-04,2.483797e-05,6.700932,4.472988e-11
SaleCondition.new,4.055894e-01,6.653294e-02,6.096069,1.858089e-09
Neighborhood.new,1.263385e-01,2.544617e-02,4.964930,8.778951e-07
is_full_functional,7.884250e-02,1.805382e-02,4.367081,1.463685e-05
ScreenPorch,3.090737e-04,7.238177e-05,4.270049,2.244068e-05


In [93]:
model.lm <- lm(sale_price_log ~ ., data=trainset.ready %>% select(-has_pool))

y_predicted <- predict(model.lm, testset.ready) %>% as.vector
y_actual <- testset %>% mutate(sale_price_log = log(SalePrice)) %>% select(sale_price_log) %>% `[[`(1)
rmse(y_predicted, y_actual)

sp_predicted <- y_predicted %>% exp
sp_actual <- testset %>% select(SalePrice) %>% `[[`(1)
mean(100 * abs(sp_actual - sp_predicted) / sp_actual)

glance(model.lm)
tidy(model.lm) %>% arrange(p.value) %>% filter(p.value < 0.1)

Warning message in predict.lm(model.lm, testset.ready):
“prediction from a rank-deficient fit may be misleading”

[1] 0.1758848

[1] 11.68465

r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
0.9370247,0.9298138,0.1057985,129.9454,0,76,644.8659,-1135.732,-781.962,7.331626,655


term,estimate,std.error,statistic,p.value
X2ndFlrSF,2.534780e-04,2.344324e-05,10.812410,3.476620e-25
OverallQual,5.446424e-02,6.249640e-03,8.714781,2.377568e-17
X1stFlrSF,2.400520e-04,2.849345e-05,8.424816,2.287399e-16
OverallCond,4.068455e-02,5.250685e-03,7.748428,3.555334e-14
MSZoning.new,2.521952e-01,3.735166e-02,6.751915,3.219963e-11
BsmtFinSF1,1.660724e-04,2.477685e-05,6.702724,4.416976e-11
SaleCondition.new,4.060677e-01,6.645755e-02,6.110181,1.707772e-09
Neighborhood.new,1.261320e-01,2.541443e-02,4.963006,8.859959e-07
is_full_functional,7.898600e-02,1.803169e-02,4.380399,1.379104e-05
ScreenPorch,3.086220e-04,7.230719e-05,4.268206,2.261704e-05
